In [1]:
import re

# Punctuation list
punctuations = re.escape('!"#%\'()*+,./:;[]<=>?@\\^_`{|}~')

# ##### #
# Regex #
# ##### #
re_remove_brackets = re.compile(r'\{.*\}')
re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
re_transform_numbers = re.compile(r'\d', re.UNICODE)
re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
# Different quotes are used.
re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
re_tree_dots = re.compile(u'…', re.UNICODE)
# Differents punctuation patterns are used.
re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
                       (punctuations, punctuations), re.UNICODE)
re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
                         (punctuations, punctuations), re.UNICODE)
re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
re_changehyphen = re.compile(u'–')
re_doublequotes_1 = re.compile(r'(\"\")')
re_doublequotes_2 = re.compile(r'(\'\')')
re_trim = re.compile(r' +', re.UNICODE)

In [2]:
def limpar_texto(text):
    """Apply all regex above to a given string."""
    text = text.lower()                                   # transformar em minusculas
    text = text.replace('\xa0', ' ')                      # non-breaking space >> space
    text = re_tree_dots.sub('...', text)                  # caracter ... >> ...
    text = re.sub('\[\.\.\.\]|\(\.\.\.\)', 'XXSALTO ', text)
    text = re.sub('(regimento interno|ri)/tcu', 'regimento interno do tcu', text)
    text = re.sub('constituição federal/1988', 'constituição federal', text)    
    text = re.sub('da cf', 'da constituição federal', text)    
    text = re.sub('c/c', 'combinado com', text)
    text = re.sub('[ve]\.g\.', 'por exemplo', text)
    text = re.sub('ex vi (d[aeo])', r'por força \1', text)    
    text = re.sub('ex vi', 'por força de', text)    
    text = re.sub('sras\.', 'senhoras', text)
    text = re.sub('sra\.', 'senhora', text)
    text = re.sub('srs\.', 'senhores', text)
    text = re.sub('sr\.', 'senhor', text)
    text = re.sub('/(ac|al|ap|am|ba|ce|df|es|go|ma|mt|ms|mg|pa|pb|pr|pe|pi|rj|rn|rs|ro|rr|sc|sp|se|to)(\W)', r' XXUF\2', text)
    #text = re.sub('\.\.\.', '', text)                     # remover ...
    text = re_remove_brackets.sub('', text)               # remover tudo entre chaves {}
    text = re_changehyphen.sub('-', text)                 # travessao >> hifen
    text = re_remove_html.sub(' ', text)                  # remover tudo entre <> (tags html)
    text = re_transform_numbers.sub('0', text)            # digito >> 0
    text = re.sub('#0+;', ' ', text)                       # remover esse código
    text = re_transform_url.sub('XXURL', text)            # urls >> token "XXURL"
    text = re_transform_emails.sub('XXEMAIL', text)       # emails >> token "XXEMAIL"
    text = re_quotes_1.sub(r'\1"', text)                  # aspas no início >> "
    text = re_quotes_2.sub(r'"\1', text)                  # aspas no fim >> "
    text = re_quotes_3.sub('"', text)                     # aspas no meio >> "
    text = re.sub('"', '', text)                          # remover "
    text = re_dots.sub('.', text)                         # pontuações . ! ? .. ... >> .
    text = re.sub('(\(|\[)(responsáve(l\s*\d?|is)|interessad[ao]s?|representantes?|recorrentes?)(\)|\])', 'XXOMISSIS', text)
    text = re.sub('(\(|\[)(omissis|ex-servidor(es)?|empresas?|segurados?)(\)|\])', 'XXOMISSIS', text)
    text = re.sub('0?0/0?0/00(00)?', 'XXDATA', text)
    text = re.sub('tc-000.000/0000-0', 'XXPROCESSO', text)
    text = re.sub('(nº)?\s+(0(\.|-|/)?){5,}0', 'XXNUMERACAO', text)
    text = re.sub('XXNUMERACAO XXUF', 'XXNUMERACAO', text)    
    text = re.sub('(0{1,3}(\.000)*|0+)*/00(00)?', 'XXNUMERACAO', text)
    text = re.sub('\(0+(,0+)?%\)|0+(,0+)?%', 'XXPERCENTUAL', text)
    text = re.sub('(0{1,3}(\.000)*|0+)-\s*(ac|al|ap|am|ba|ce|df|es|go|ma|mt|ms|mg|pa|pb|pr|pe|pi|rj|rn|rs|ro|rr|sc|sp|se|to)', ' XXNUMERACAO', text)
    text = re.sub('(nº\s*)?XXNUMERACAO\s*-\s*tcu', ' XXNUMERACAO', text)
    text = re.sub('(0ª ?|primeira |segunda )câmara', 'XXCAMARA', text)
    text = re.sub('(XXNUMERACAO[,;]?\s+)+XXNUMERACAO\s*(-|,\s*(ambos\s+)?d(a|o|este)|\s)\s*(XXCAMARA|plenário|p)', 'XXACORDAOS', text)
    text = re.sub('XXNUMERACAO\s*(-|,\s*d[ao]|\s)\s*(XXCAMARA|plenário|p)', 'XXACORDAO', text)
    text = re.sub('XXNUMERACAO\s+e\s+XXNUMERACAO\s*,\s*(ambos\s+)?d(a|o|este)\s*(XXCAMARA|plenário|p)', 'XXACORDAOS', text)    
    text = re.sub('((XXACORDAOS?|XXNUMERACAO)(,|;|\s+e)?\s+)+XXACORDAOS?', 'XXACORDAOS', text)
    text = re.sub('(lei|decreto) (nº )?XXNUMERACAO(, de XXDATA)?', r'\1 XXNORMA', text)
    text = re.sub('art(\.|igo) (0º|0+) da lei XXNORMA', 'artigo XXITEM da lei XXNORMA', text)
    text = re.sub('(0+) da lei XXNORMA', 'XXITEM da lei XXNORMA', text)
    text = re.sub('art(\.|igo) (0º|0+)(-[a-z])?', 'artigo XXITEM', text)
    text = re.sub('art(s\.|igos) ((0º|0+)(-[a-z])?\s*[,e])+ (0º|0+)(-[a-z])?', 'artigos XXITEM e XXITEM', text)
    text = re.sub('art(s\.|igos) (0º|0+)(-[a-z])?', 'artigos XXITEM', text)
    text = re.sub('(parágrafo|§)\s*(0º|0+)', 'parágrafo XXITEM', text)
    text = re.sub('inciso (i?v|v?(iii|ii|i))', 'inciso XXITEM', text)
    text = re.sub('alínea [a-z]', 'alínea XXITEM', text)
    text = re.sub('alíneas [a-z] e [a-z]', 'alíneas XXITEM', text)
    text = re.sub('(0º|0+), (inciso|parágrafo|§) XXITEM', r'XXITEM, \2 XXITEM', text)
    text = re.sub('0+, caput', 'XXITEM, caput', text)
    text = re.sub('r\$ (0{1,3}([.,]000)*|0+)[.,]00( \(.+\sreais\))?', 'XXQUANTIA', text)
    text = re.sub('(desde|em|de)\s0000', r'\1 XXDATA', text)
    text = re.sub('(\W)(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)', r'\1 XXMES', text)
    text = re.sub('XXMES\s+de\s+XXDATA', 'XXDATA', text)
    text = re.sub('0000\s*(-|a)\s*0000', 'XXPERIODO', text)
    text = re.sub('XXDATA\s*(-|a)\s*XXDATA', 'XXPERIODO', text)
    text = re.sub('0+\.0+\.0+\.0+\.?\s+', ' XXSUBITEM ', text)
    text = re.sub('0+\.0+\.0+\.?\s+', ' XXSUBSUBSECAO ', text)
    text = re.sub('0+\.0+\.?\s+', ' XXSUBSECAO ', text)
    text = re.sub('0+\.\s+', ' XXSECAO ', text)
    text = re.sub(' (i?v|v?(iii|ii)|i)(\s|\.|,)', r' XXNUMERACAO \3', text)
    text = re.sub('XXNUMERACAO\s+e vi(\s|\.|,)', r'XXNUMERACAO \1', text)
    text = re.sub('nº XXNUMERACAO', 'XXNUMERACAO', text)
    text = re.sub('0+\s*\(.+?\)', 'XXNUMERO', text)
    text = re.sub('(0{1,3}(\.000)*|0+)', 'XXNUMERO', text)
    text = re.sub('XXNUMERACAO\s*.XXNUMERO', 'XXNUMERACAO', text)
    text = re.sub('[a-z]+/[a-z]+/[a-z]+-[a-z]+', 'XXSIGLA', text)
    text = re.sub('XXNUMERACAO-XXSIGLA', 'XXNUMERACAO', text)
    text = re.sub('ms-(stf|sft) XXUF XXNUMERO', 'mandado de segurança stf XXNUMERACAO', text)
    text = re.sub('ms(\s+|\s*-\s*)(XXNUMERACAO|XXNUMERO)', 'mandado de segurança XXNUMERACAO', text)
    text = re.sub('súmula vinculante do (stf|sft) (no?\.? )?XXNUMERO', 'súmula vinculante do stf XXNUMERACAO', text)
    text = re.sub('súmula vinculante (n(º|o)?\.? )?XXNUMERO', 'súmula vinculante XXNUMERACAO', text)
    text = re.sub('are (n(º|o)?\.? )?XXNUMERO', 'recurso extraordinário com agravo XXNUMERACAO', text)
    text = re.sub('re (n(º|o)?\.? )?XXNUMERO', 'recurso extraordinário XXNUMERACAO', text)
    text = re.sub('ns?\.\s*(XXNUMERO|XXNUMERACAO|XXACORDAO)', r'\1', text)
    text = re.sub('n(º|s?\.|os?\.)\s*(XXNUMERO)', 'XXNUMERACAO', text)
    text = re.sub('(\s*XXNUMERACAO\s*(,|;|e))+\s+XXNUMERACAO', ' XXNUMERACAO', text)
    text = re.sub('(mandado\s+de\s+segurança\s+XXNUMERACAO\s*,\s+)+mandado de segurança XXNUMERACAO', 'mandados de segurança XXNUMERACAO', text)
    text = re.sub('ins+(XXNUMERO)', 'instrução normativa XXNUMERACAO', text)
    text = re.sub('(\w)acórdão:', r'\1 . acórdão:', text)
    #print(text)
    text = re_punctuation.sub(r'\1', text)                # pontuações . ;:""
    text = re_hiphen.sub(' - ', text)                     # colocar espacos nos tracos e pontuações
    text = re.sub('á\s?-\s?(l[ao]s?)', r'ar \1', text)
    text = re.sub('ê\s?-\s?(l[ao]s?)', r'er \1', text)
    text = re.sub('ô\s?-\s?(l[ao]s?)', r'or \1', text)
    text = re.sub('-\s?(lhes?|[ln]?[ao]s?|se)', r' \1', text)
    text = re_punkts.sub(r'\1 \2 \3', text)
    text = re_punkts_b.sub(r'\1 \2 \3', text)
    text = re_punkts_c.sub(r'\1 \2', text)
    text = re_doublequotes_1.sub('\"', text)
    text = re_doublequotes_2.sub('\'', text)
    text = re.sub('(\.|:|;|-|/)', r' \1 ', text)                         # incluir espacos
    text = re.sub('\(', ' ( ', text)                         # incluir espacos
    text = re.sub('\)', ' ) ', text)                         # incluir espacos
    text = re_trim.sub(' ', text)                         # remover espacos extras
    return text.strip()

In [3]:
22

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_EXCERTO,TEXTO_EXCERTO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995438,Voto:Cuidam os autos de tomada de contas espec...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025603,Voto:Cuidam os autos de Solicitação do Congres...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455375,Relatório:Trata-se de embargos de declaração o...,Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773747,Voto:8. Em relação a outros processos judiciai...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773403,Voto:11. Relativamente ao ato envolvendo a Sra...,Acórdão 55/2016 - PL,Representação


In [4]:
from cogroo_interface import Cogroo
cogroo = Cogroo.Instance()

In [7]:
df['EXCERTO_LEMATIZADO'] = df['TEXTO_EXCERTO'].apply(limpar_texto).apply(cogroo.lemmatize)

In [8]:
df.to_csv('../dados/excertos_lematizados.csv', index=False, sep = '|', columns = ['COD', 'DESCR_AREA', 'EXCERTO_LEMATIZADO'])